<a href="https://colab.research.google.com/github/dandoreyrodriguez/Bayesian_Econometrics/blob/main/BE_Koop_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Class 1: in Bayesian Econometrics**

## **Motivation**

This is the first in a series of notebooks which document my journey learning Bayesian econometrics. I use ChatGPT 4o and Gary Koop's 2003 "Bayesian Econometrics". It is intended for people who, like me, have zero background in Bayesian statistics but have a background in statistics/econometrics. The point of this is not to mimic a textbook, but to show how I slowly began to understand things. My hope is that I can closely track the difficulties other new students encounter, and that the reader can learn from them. The reason I figured I had to learn Bayesian statistics is that Bayesian techniques yield practical advantages in the context of models many economists care about (e.g. dynamic macro models or state space time series models), though I do not understand how or why.

Some Bayesian advantages include:

- Less over-fitting in small samples
- Can help avoid computational issues traditional (frequentist) methods run into in non-linear contexts
- More explicit treatment of parameter uncertainty
- Use nice properties of simulation methods, like Monte Carlo methods.

I leverage the power of python notebooks to do an important part of Bayesian econometrics---computation. A large goal of this series is therefore to help new students learn to use Bayesian tools in python, chiefly `pyMC`.

I go over the necessary Bayesian theory first and to get to applications as soon as possible. However, I am a firm believer that practitioners should not use techniques whose statistical properties they do not understand. So, I plan to give the theory a thorough treatment. If the reader struggles with this, I hope they may find comfort in knowing that I did too.

Happy learning.

## **A Bayesian Interpretation of Probability**

The traditional interpretation of probability is that it represents a long-run frequency. This school of statistical thought is called *frequentist* statistics. For example, the probability of flipping a coin on heads is the number of times it lands on heads divided by the number of times you flip the coin (often called trials) as the number of trials approaches infinity. This emphasises the objective nature of probability; that probability is a byproduct of the system being studied.

An alternative approach to statistics is the *Bayesian* approach. It interprets probability as a subjective degree of belief of an event given current information. The task then becomes updating one's beliefs as new information is observed.

However, Bayesian statistics does not discard any of the rules of probability. The fundamental axioms of probability still hold (i.e. the set-theoretic foundations laid down by Kolmogorov). From an econometric point of view, whether *frequentist* or *Bayesian*, the goal is still the same. We want to use observed data to learn about parameters, models, and make out-of-sample predictions.

## **Some Bayesian Basics**

This section covers some fundamental Bayesian basics. Its main purpose is to familiarise the reader with the objects of interest in Bayesian statistics.

In this section I will not be formal and will, to an extent, abuse notation. This is to emphasise Bayesian ideas.

The fundamental object in Bayesian statistics is $P(\boldsymbol{\theta} | \boldsymbol{D})$, the probability distribution (a measure of belief) over parameters given both the observed data and some assumed model. It is known as the *posterior distribution*. Some notation:

- $\boldsymbol{D}$ is a matrix (or vector) of data
- $\boldsymbol{\theta}$ is a vector of parameters for a particular model.

By the definition of a probability density function, all the same arguments pass treating $P(\boldsymbol{\theta} |  \boldsymbol{D})$ as a density function instead of a mass function.

The key aim in Bayesian statistics is to learn about the *posterior distribution* from the other objects. With knowledge of the *posterior distribution* we can compute $E[\boldsymbol{\theta} |  \boldsymbol{D}]$, which is a logical estimate of $\boldsymbol{\theta}$: we can also completely characertise the uncertainty around this estimate.

Unsurprisingly, Bayes' rule is at the heart of how *Bayesians* learn about the posterior:

$$P(\boldsymbol{\theta} | \boldsymbol{D})  = \frac{ P(\boldsymbol{D}|\boldsymbol{\theta}) P(\boldsymbol{\theta} ) }{P(\boldsymbol{D})}$$

So, having observed the data, we can use Bayes' rule to update our beliefs about the model parameters.

What are the right hand side objects called?
- $P(\boldsymbol{D}|\boldsymbol{\theta})$ is a *likelihood function*, a familiar object!
- $P(\boldsymbol{\theta} )$ is called the *prior distribution*. This distribution reflects the intial uncertainty over parameter values
- $P(\boldsymbol{D})$ is called the *marginal likelihood*, or *evidence*. It ensures a valid probability distribution (is a normalising constant).

The reason why Bayesian methods rely so much on computation is because the *marginal likelihood* is usually impossible solve for analytically. So, computational methods have been developed to estimate $P(\boldsymbol{\theta} | \boldsymbol{D})$ without an expression for $P(\boldsymbol{D})$.

To be more precise, in a continuous case, $P(\boldsymbol{D})$ involves evaluating the following integral:

$$P(\boldsymbol{D})= \int_{\boldsymbol{\theta} \in \boldsymbol{\Theta}} P(\boldsymbol{D} | \boldsymbol{\theta} ) P(\boldsymbol{\theta})d \boldsymbol{\theta} $$

More often than not, this object is not available analytically. For this reason, Bayesians have developed computational ways to estimate $P(\boldsymbol{\theta} | \boldsymbol{D})$. This usually involves working with the *posterior distribution* in the following form:

$$P(\boldsymbol{\theta} | \boldsymbol{D}) \propto P(\boldsymbol{D}|\boldsymbol{\theta}) P(\boldsymbol{\theta} ) $$

That is, combining the likelihood function and prior completely describe the shape of the posterior. Then, using computational procedures, one can estimate the posterior and the scaling necessary to ensure the posterior is a valid probability distribution.

According to Koop, one of the main advantages of the Bayesian framework is its generality. He says there are always familiar steps:

1. Choose a *prior distribution* and *likelihood function*
2. Obtain the *posterior distribution*
3. Do whatever statistical inference you like!

One of the biggest critiques of Bayesian methods surrounds the *prior distribution*. People say that it admits arbitrariness. I will keep this in mind as I proceed.

To learn how to Bayesian econometrics, I will start by learning how a Bayesian would attack a normal regression model---familiar territory to statisticians and econometricians alike.

## **Linear Regression**

A linear regression model is the most convenient/basic model to think about. I start with this example because it is what I, and most others, know best.

Suppose that $\boldsymbol{y}$ is a $n \times 1 $ vector of data and $\boldsymbol{X}$ is a $n \times k$ design matrix. The $i$th row of this matrix is $\boldsymbol{x}_i'$, where $\boldsymbol{x}_i$ is a $k\times1$ vector which stores each of the features, or independent variables, for that observation. A normal linear regression model is:

$$\boldsymbol{y}= \boldsymbol{X}\boldsymbol{\beta}+\boldsymbol{\epsilon}, \, \boldsymbol{\epsilon} \overset{\text{i.i.d}}{\sim} \text{N}(\boldsymbol{0}, h^{-1} \boldsymbol{I})$$

This is the standard homoskedastic, serially-uncorrelated-error case.  $h = \sigma^{-2}$ is a measure of "tightness" of errors. $\boldsymbol{\beta}$ is the $k \times 1$ vector of parameters of interest. With an estimate of this vector, one can attempt out-of-sample prediction. A *frequentist* treats $\boldsymbol{\beta}$ is constant (i.e. given), but unknown.

But how would a *Bayesian* learn about $\boldsymbol{\beta}$?

Start by considering the likeliood function. In this model, observations are normally distributed:

$$\boldsymbol{y}|\boldsymbol{X},\boldsymbol{\beta},h \overset{\text{i.i.d}}{\sim} \text{N}(\boldsymbol{X}\boldsymbol{\beta}, h^{-1} \boldsymbol{I})$$

The likelhood function is simlpy the joint probability density function of the data.

$$f(\boldsymbol{y}|\boldsymbol{X}, \boldsymbol{\beta},h) = (2 \pi)^{-\frac{n}{2}} h^{\frac{n}{2}}  \text{exp}\bigg( -\frac{h}{2}(\boldsymbol{y}-\boldsymbol{X}\boldsymbol{\beta})' (\boldsymbol{y}-\boldsymbol{X}\boldsymbol{\beta})\bigg) $$
